# Center, Spread, Normal Distribution

Complete this problem set with your group (up to 4 students). You can either hand-write your work and submit a hard copy, or go to File -> Download as HTML to generate an HTML file, then either submit it online or as a hard copy. Each group only needs to submit one copy and will receive the same grade. 

This team homework is based on Lecture 25: Center and Spread, Lecture 26: Normal Distribution and Lecture 27: Sample Means. 

**Helpful Resource:**
- [Python Reference](http://data8.org/sp22/python-reference.html): Cheat sheet of helpful array & table methods used in Data 8!

**Recommended Readings**: 
- [Mean and Median](https://inferentialthinking.com/chapters/14/1/Properties_of_the_Mean.html#)
- [Standard Deviation](https://inferentialthinking.com/chapters/14/2/Variability.html)
- [Central Limit Theorem for proportions](https://inferentialthinking.com/chapters/14/4/Central_Limit_Theorem.html)
- [Central Limit Theorem for Sample Means](https://inferentialthinking.com/chapters/14/5/Variability_of_the_Sample_Mean.html)

In [ ]:
# These lines import the Numpy and Datascience modules.
import numpy as np
from datascience import *
from scipy import stats
from numpy import sqrt
# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore')

### Mean, Median, Standard Deviation

Find the mean, median and standard deviation of the following data set: 19, 19, 20, 22. Do your work either on paper or using the Python command line *without* the use of ```np.median``` and ```np.sd``` 

Note: the formula for standard deviation is $SD = \sqrt{\frac{\sum_i (x_i - \bar{x})^2}{n}}$

How would the addition of an extreme value affect the mean, median and standard deviation? (the statistic is said to be either _sensitive_ or _resistant_ to outliers)

Use the student data and `np` functions (`np.average`, `np.median`, `np.std`) to answer questions about the center and spread. 

In [ ]:
students = Table().read_table('student_data.csv')
students.hist('HEIGHT', group='SEX')

Find the mean, median and the standard deviation of female and male students, respectively. 

Examine the tallest female student and the tallest male student. Relatively speaking, who is taller in terms of the standard units (z score)? 

Identify two variables, one showing a left-skewed distribution, and the other a right-skewed distribution. What will be the result from the following command? 

In [ ]:
# Identify a left or right-skewed disribution from the student data
x = students.column(...)
(np.count_nonzero(x > np.average(x)) / len(x)) > 0.5


### Estimating Standard Deviation 

In addition to observing the inflection point on the histogram, another method of estimating standard deviation from the histogram is to use the interval [a, b] that identifies the middle 95% of the total area, and use $SD \approx \frac{(b-a)}{4}$. This works best when the distribution was approximately bell shaped and symmetric. 

Use this method to estimate the standard deviation from the following distributions. 

<img src="bootstrap_mean.GIF" width=600>
<img src="bootstrap_proportion.png" width=600>

In [ ]:
# Enter your estimate of the standard deviation 




Based on the ```percentile``` function and the estimation formula $SD \approx \frac{(b-a)}{4}$, estimate the standard deviation of female and male height, respectively. How does your results compare with `np.std`?

### Normal Distribution 

Be sure to import the `scipy.stats` package done in the beginning of the notebook. We will use `plot_normal_cdf` and the `stats.norm.cdf` functions together.

In [ ]:
plot_normal_cdf(lbound = -1, rbound = 1)

You can also draw the curve manually by using the explicit function: 

In [ ]:
from numpy import exp, pi
x = np.arange(-3, 3, 0.1)
y = (1/sqrt(2*pi))*exp(-(x**2)/(2))
plots.plot(x,y)

Use `stats.norm.cdf` to find the area between z = -1.5 and 0.5 

Find the area to the right of z = 3

Use the mean and standard deviation of the female students and use ```plot_normal_cdf(rbound, lbound, mean, sd)``` to create a similar graph as the one shown for $P(-1<Z<1)$, but in different scales. 

Examine the following code. What does the `stats.norm.ppf` function do? Describe it in your own words. 

In [ ]:
a = stats.norm.ppf(0.05)
b = stats.norm.ppf(0.95)
stats.norm.cdf(b) - stats.norm.cdf(a)

Use `stats.norm.ppf` to identify z-scores that mark the middle 95% of the area (e.g. $P(-a < Z < a) = 0.95$)) under the standard normal distribution. How are they related to the formula for estimating the standard deviation? 

Using the same height data, we can draw two normal curves with the respective mean and standard deviation of male and female students and superimpose them on the original histogram. 

In [ ]:
students = Table().read_table('student_data.csv')
students.hist('HEIGHT', group='SEX')

# keep these x ranges 
x = np.arange(60, 75, 0.1)
# replace the means and standard deviations with the values found from Table.group() 
yf = stats.norm.pdf(x, 62, 1)
ym = stats.norm.pdf(x, 72, 4)

plots.plot(x, yf, x, ym)

### Central Limit Theorem for Sample Means

Treat the student data as the population, and simulate drawing samples of 4 from the population. Be sure to use sampling with replacement. Show the histogram of your sample means and find its mean and standard deviation. 

In [ ]:
students.hist('AGE')
all_ages = students.column('AGE')
x = np.arange(min(all_ages), max(all_ages), 0.1)
y = stats.norm.pdf(x, np.average(all_ages), np.std(all_ages))
plots.plot(x, y)

In [ ]:
# this draws the sample of 4 with replacement. 
# Now construct a sampling distribution of means by repeating sampling 
students.sample(4)

Now repeat your experiment by increasing the sample size to 16. What happened to the mean and standard deviation of these sample means? 

The following code helps us compare the sampling distribution from different sample sizes (n) 

In [ ]:
def sampling_distribution_means(table, col_label, n, iter):
    results = make_array()
    for i in np.arange(iter):
        sample_mean = np.average(table.sample(n).column(col_label))
        results = np.append(results, sample_mean)
    print("Mean of Sample Means of n=", n, ":", np.average(results))
    print("StDev of Sample Means of n=", n, ":", np.std(results))
    return results

def CLT_demo(table, col_label):
    nreps = 5000
    results_n1 = sampling_distribution_means(table, col_label, 1, nreps)
    results_n4 = sampling_distribution_means(table, col_label, 4, nreps)
    results_n16 = sampling_distribution_means(table, col_label, 16, nreps)
    nsize = np.concatenate((np.ones(nreps), np.ones(nreps)*4, np.ones(nreps)*16))
    results = np.concatenate((results_n1, results_n4, results_n16))
    results_table = Table().with_columns("Sample Size", nsize, "Sample Means", results)
    return results_table

results_table = CLT_demo(students, 'AGE')
results_table.hist(group="Sample Size")

means_table = results_table.group('Sample Size', np.average)
sd_table = results_table.group('Sample Size', np.std)

means_table, sd_table
x = np.arange(15, 50, 0.1)
y1 = stats.norm.pdf(x, means_table.column(1).item(0), sd_table.column(1).item(0))

y4 = stats.norm.pdf(x, means_table.column(1).item(1), sd_table.column(1).item(1))
y16 = stats.norm.pdf(x, means_table.column(1).item(2), sd_table.column(1).item(2))

plots.plot(x, y1, x, y4, x, y16)

How does your finding compare with the predictions of [Central Limit Theorem](https://inferentialthinking.com/chapters/14/5/Variability_of_the_Sample_Mean.html) ? 

Three elements of the Central Limit Theorem: 
- For a fixed sample size (larger is better), the distribution of all sample means is approximately normal. 
- $\text{Mean of all possible sample means} = \text{Population Mean}$
- $\text{SD of all possible sample means} = \frac{\text{Population SD}}{\sqrt{\text{sample size}}}$

### Central Limit Theorem for Sample Proportions

The sample proportions can be seen as the mean of a binary variable (coin toss), where $X=1$ represents the heads of a coin toss, and $X=0$ otherwise. It can be shown that if $P(X=1)=p$, then the mean and SD for the binary variable is $p$ and $\sqrt{p(1-p)}$, respectively. We can verify these formulas through the simulation below. 

In [ ]:
p = 0.26
model = make_array(p, 1-p)
results = make_array()
# we can simulate tossing 1 coin through sample_proportions by using n=1
for i in np.arange(10000):
    prop = sample_proportions(1, model).item(0)
    results = np.append(results, prop)
# the mean and SD of this binary variable is p and sqrt(p*(1-p)), respectively. 
print('Predicted mean=', p, ', SD=', sqrt(p*(1-p)))
np.average(results), np.std(results)


Returning to the Alabama jury example, we can use the underlying parameter $p$ to generate a sampling distribution of sample proportions based on sample size $n$. Since each sample proportion can be seen as the mean of $n$ binary variables, Central Limit Theorem here predicts that: 

$$ \text{Mean of Sample Proportions} = \text{Mean of Coin Toss} = p$$
$$ \text{SD of Sample Proportions} = \frac{\text{SD of Coin Toss}}{\sqrt{n}} = \frac{\sqrt{p(1-p)}}{\sqrt{n}} = \sqrt{\frac{p(1-p)}{n}}$$

In [ ]:
# here we repeat the simulation of sampling 100 jurors from the population where p=0.26
p = 0.26
model = make_array(p, 1-p)
n = 100
results = make_array()
# here we use the sample proportion as the statistic to make it comparable to CLT
for i in np.arange(10000):
    prop = sample_proportions(n, model).item(0)
    results = np.append(results, prop)

Now we can compare the empirical distribution of sample proportion with the prediction of the Central Limit Theorem. 

In [ ]:
from math import sqrt
#computing the SD for the sample proportion according to CLT
sigma = sqrt(p*(1-p)/n)
x = np.arange(p-4*sigma, p+4*sigma, sigma/10)
#The mean of the sample proportion is p, and SD was sqrt(p*(1-p)/n)
y = stats.norm.pdf(x, p, sigma)
Table().with_column('Sample proportions', results).hist()
plots.plot(x, y)

The normal curve provides the exact p-value based on the Central Limit Theorem. To see an example, based on the null hypothesis that the jury panel accurrately represents the juror population, compare the p-value for a jury panel that contains 15% black jurors obtained through simulation, and one that is based on the normal curve. (Note: you need to convert the test statistic to a Z-score, and use ```plot_normal_cdf``` above to compute the exact p-value.)

In [ ]:
# approximate p-value by simulation


# exact p-value by CLT



Here we see that what we have done previously in hypothesis testing was a simulation that was supported by the Central Limit Theorem, and the approximate p-value gets better as the number of simulations increases. 